# Combined shelft exit location files

In [10]:
# Importing the relevant modules. 
import numpy as np
import xarray as xr
import pandas as pd
from glob import glob
import random
import gsw
from datetime import datetime

In [11]:
# define particle depths (or densities) of interest. 
zstart = 27.8
zend = 30

In [14]:
# Define offshore count files
datadir = '/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/'
files = sorted(glob(datadir+'OffshoreParticles_1x0.5degree_file-*_{}-{}sigma0.nc'.format(zstart, zend)))
#files = files[0:-1]
print(len(files))
files

73


['/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-00_27.8-30sigma0.nc',
 '/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-01_27.8-30sigma0.nc',
 '/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-02_27.8-30sigma0.nc',
 '/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-03_27.8-30sigma0.nc',
 '/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-04_27.8-30sigma0.nc',
 '/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-05_27.8-30sigma0.nc',
 '/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-06_27.8-30sigma0.nc',
 '/g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-07_27.8-30sigma0.nc',
 '/g/data/e14/hd4873/runs/parcels/output

In [16]:
for i in range(len(files)):
    print (i, files[i])
    
    ds = xr.open_dataset(files[i])
    
    if i == 0:
        ds_comb = ds.copy(deep=True)
        ds_comb['count'][:,:] = np.nan_to_num(ds_comb['count'])
        ds_comb['count_norm'][:,:] = np.nan_to_num(ds_comb['count_norm'])
        ds_comb['count_transnorm'][:,:] = np.nan_to_num(ds_comb['count_transnorm'])
        ds_comb['count_trans'][:,:] = np.nan_to_num(ds_comb['count_trans'])
    else:
        # error checking
        j = random.randint(0, 179)
        k = random.randint(0, 25)
        val1 = np.nan_to_num(ds_comb['count_transnorm'][j,k])
        val2 = np.nan_to_num(ds['count_transnorm'][j,k])
        sumvals = val1+val2
        ds_comb['count'][:,:] = np.nan_to_num(ds_comb['count']) + np.nan_to_num(ds['count'])
        ds_comb['count_norm'][:,:] = np.nan_to_num(ds_comb['count_norm']) + np.nan_to_num(ds['count_norm'])
        ds_comb['count_transnorm'][:,:] = np.nan_to_num(ds_comb['count_transnorm']) + np.nan_to_num(ds['count_transnorm'])
        ds_comb['count_trans'][:,:] = np.nan_to_num(ds_comb['count_trans']) + np.nan_to_num(ds['count_trans'])
        # error check
        if ds_comb['count_transnorm'][j,k].values != sumvals:
            print("ERROR: Summation is not corrent")
            print(i, j, k, 'value 1',val1, 'value 2', val2, 'sum:', sumvals, 'dataset sum:', ds_comb['count_transnorm'][j,k].values)
            break
            
            
ds_comb['count'] = ds_comb['count'].where(ds_comb['count']>0, np.nan)
ds_comb['count_norm'] = ds_comb['count_norm'].where(ds_comb['count_norm']>0, np.nan)       
ds_comb['count_transnorm'] = ds_comb['count_transnorm'].where(ds_comb['count_transnorm']>0, np.nan) 
ds_comb['count_trans'] = ds_comb['count_trans'].where(ds_comb['count_trans']>0, np.nan)  
# rename count variable to avoid bound method issue
ds_comb = ds_comb.rename({'count': 'count_tot'})

0 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-00_27.8-30sigma0.nc
1 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-01_27.8-30sigma0.nc
2 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-02_27.8-30sigma0.nc
3 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-03_27.8-30sigma0.nc
4 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-04_27.8-30sigma0.nc
5 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-05_27.8-30sigma0.nc
6 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-06_27.8-30sigma0.nc
7 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-07_27.8-30sigma0.nc
8 /g/data/e14/hd4873/runs/parcels/output/AntConn/data/of

In [17]:
outfile = datadir+'OffshoreParticles_1x0.5degree_{}-{}sigma0.nc'.format(zstart, zend)
#print(outfile)

# Save to file
print("Saving to netCDF file:", outfile)
encod={}
for var in ds_comb.data_vars:
    encod[var]={'zlib':True}
ds_comb.to_netcdf(outfile)

Saving to netCDF file: /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_27.8-30sigma0.nc


In [18]:
# Now remove the individual files

! rm /g/data/e14/hd4873/runs/parcels/output/AntConn/data/offshore/OffshoreParticles_1x0.5degree_file-*_*sigma0.nc